In [ ]:
# Global Variables Set by User **change these values before running script**

# URL of the portal with the template content
PROJECT_URL = "https://envisioning.maps.arcgis.com"

# Portal Username for Log-on (needs to be admin)
PROJECT_USERNAME = "GBushongENV"

# URL of the portal that will be customized and have users added
DELIVERY_URL = "https://esrienergy.maps.arcgis.com"

# Delivery username for log-on (needs to be admin)
DELIVERY_USERNAME = "PDOAdmin"

# delivery org UX Component locations - local full path
FOLDER = "/Users/grac9792/Downloads/Sample_Config" 

# delivery groups to share with *ALL* users
GROUP_IDS = ["4d7ff4f81d6340428ef290b7de801204"]

# featured group to display on delivery home page
FEATURED_GROUP_ID = "4f4fcac023dc430294cea226231ab448"

# filenames of UI components and user file, which are located in FOLDER
THUMBNAIL_FILENAME = "thumbnail.png"
FOOTER_FILENAME = "footer.txt"
BACKGROUND_FILENAME = "background.png"
BANNER_FILENAME = "banner.png"
DESCRIPTION_FILENAME = "description.txt"
USERS_FILENAME = "users.csv"

# message that will be sent to all users when they receive the invitations to join the group
MESSAGE = "You are invited to join a group!"

# how long the invites will be valid for: either '1 Day', '3 Days', '1 Week', or '2 Weeks'
EXPIRATION = '1 Day'

In [ ]:
# import libraries
from arcgis.gis import GIS
from arcgis.gis import Group
from arcgis.gis import UserManager
import os
import csv

In [3]:
# get UI component filepaths
THUMBNAIL = os.path.join(FOLDER, THUMBNAIL_FILENAME)
FOOTER = os.path.join(FOLDER, FOOTER_FILENAME)
BACKGROUND = os.path.join(FOLDER, BACKGROUND_FILENAME)
BANNER = os.path.join(FOLDER, BANNER_FILENAME)
DESCRIPTION = os.path.join(FOLDER, DESCRIPTION_FILENAME)
USERS = os.path.join(FOLDER, USERS_FILENAME)
USER_FIELDS = ["username", "password", "firstname", "lastname", "email", "role", "groups", "level"]

In [4]:
# connect to project and delivery orgs
print("Connecting to {} with username {}...".format(PROJECT_URL, PROJECT_USERNAME))
project = GIS(PROJECT_URL, PROJECT_USERNAME)

print("Connecting to {} with username {}...".format(DELIVERY_URL, DELIVERY_USERNAME))
delivery = GIS(DELIVERY_URL, DELIVERY_USERNAME)

Enter password: ········
Enter password: ········


In [15]:
# Creation of functions to be used later
def optional_args_handler(key, user, default=None):
    """Sets optional user creation arguments appropriately"""
    try:
        var = None
        if key in user:
            var = user[key]
        else:
            if default: var = default
        return var
    except Exception as e:
        print("Optional args handler failed with args {}, {}, and {}: {}".format(key, user, default, e))

def add_user(user, origin, target, groups=[]):
    """Add user to the gis and to specified groups
    * Abstraction for creating from dict such as with csv

    args:
    user -- a dictionary containing user fields, see fields:
        http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.UserManager.create
    origin -- the project org, where users will be invited to groups
    target -- the delivery org, where users are added
    groups -- (optional) destination groups, compliments those in dict
    """
    try:
        print("INFO: Creating user {}".format(user["username"]))
        role = optional_args_handler('role', user, 'org_user') # default is org_user
        level = optional_args_handler('level', user, '2') # default is 2
        
        # create array of groups for user
        user_groups = []
        for grp in groups:
            user_groups.append(grp)

        group_str = user.pop('groups', None)
        if group_str:
            group_list = group_str.split(",")
            for g in group_list:
                group_search = Group(origin, g)
                if group_search.title:
                    user_groups.append(group_search)
                else:
                    print("Could not find group id {}".format(g))
        print("User will be invited to join: {}".format([g.title for g in user_groups]))
        
        result = target.users.create(username=user['username'], password=user['password'], firstname=user['firstname'], 
                                  lastname=user['lastname'], email=user['email'], role=role, level=level)

        # create users returns None if it was unsuccessful
        if not result:
            print("Did not create user: Check username {}".format(user['username']))
            return

        # Add user to groups
        print("Adding to groups")
        for g in user_groups:
            try:
                res = g.invite_by_email(user['email'], '{} {},\n{}'.format(user['firstname'], user['lastname'], MESSAGE), expiration=EXPIRATION)
                if not res['success'] or res['success'] == False:
                    print("Failed to invite user {} to group {}: {}".format(user['username'], g.title, res))
                else:
                    print("Invited user {} to group {} by email".format(user['username'], g.title))
                #print("Result of sending invitation email: {}".format(res))
            except Exception as e:
                print("ERR: Could not add user {} to group {}: {}".format(user['username'], g, e))
        return result
    except Exception as e:
        print("ERR: Could not create user {}: Add User failed with error {}".format(user['username'], e))

def add_users_csv(csv_file, origin, target, groups=None):
    """Add users from csv to gis

    args:
    csv_file -- path to csv with users to create
    origin -- project organization
    target -- delivery organization where users are added
    groups -- (optional) destination groups, compliments those in csv (default [])
    """
    try:
        results = []
        with open(csv_file, 'r') as users_csv:
            users = csv.DictReader(users_csv)
            for user in users:
                result = add_user(user, origin, target, groups=groups)
                results.append(result)
        return results
    except Exception as e:
        print("Add users csv failed with args {}, {}, {}, and {}: {}".format(csv_file, origin, target, groups, e))

In [14]:
# fetch groups where *ALL* users will be added
share_groups = tuple(Group(project, g) for g in GROUP_IDS)
print(share_groups)

# add users to the delivery org and invite them to groups in project org
add_users_csv(USERS, project, delivery, groups=share_groups)

(<Group title:"aec-test-group" owner:GBushongENV>,)
INFO: Creating user ljccstudent1
User will be invited to join: ['aec-test-group']
Adding to groups
Invited user ljccstudent1 to group aec-test-group by email

-----------------------------------------------------

INFO: Creating user ljccstudent2
User will be invited to join: ['aec-test-group', 'aec-test-group-2']
Adding to groups
Invited user ljccstudent2 to group aec-test-group by email
Invited user ljccstudent2 to group aec-test-group-2 by email

-----------------------------------------------------

INFO: Creating user ljccstudent3
User will be invited to join: ['aec-test-group']
Adding to groups
Invited user ljccstudent3 to group aec-test-group by email

-----------------------------------------------------

INFO: Creating user ljccstudent4
User will be invited to join: ['aec-test-group', 'aec-test-group-2']
Adding to groups
Invited user ljccstudent4 to group aec-test-group by email
Invited user ljccstudent4 to group aec-test-gro

[<User username:ljccstudent1>,
 <User username:ljccstudent2>,
 <User username:ljccstudent3>,
 <User username:ljccstudent4>]

In [9]:
# customize UI by setting background, banner, logo, etc.
ux = delivery.admin.ux
ux.set_background(BACKGROUND)
ux.set_banner(BANNER)
ux.set_logo(THUMBNAIL)

# read description text file and set portal description
with open(DESCRIPTION, 'r') as f:
    description = f.read()
    ux.description = description
    
# find and set featured group
if FEATURED_GROUP_ID:
    featured_group = Group(delivery, FEATURED_GROUP_ID)
    ux.featured_content = {'group': featured_group}